In [1]:
import os
import random 

import pandas as pd 
import numpy as np 
from scipy.stats import kurtosis, skew 
from scipy import stats

import matplotlib.pyplot as plt 
import seaborn as sns
# Importing librarys to use on interactive graphs
import plotly.offline as plty
from plotly import tools
import plotly.express as px
import time
from plotly.offline import init_notebook_mode, iplot, plot 
import plotly.graph_objs as go 
from sklearn import model_selection, preprocessing, metrics
import lightgbm as lgb
import json # to convert json in df
from pandas import json_normalize # to normalize the json file
from datetime import datetime
from sklearn.feature_selection import SelectKBest, chi2
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
# !unzip -q "./gdrive/MyDrive/train-255.zip"

In [4]:
df_train=pd.read_csv('./train.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning:

Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.



In [5]:
df_train.head()

,channelGrouping,date,device,fullVisitorId,geoNetwork,sessionId,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime
0,Organic Search,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",1131660440785968503,"{""continent"": ""Asia"", ""subContinent"": ""Western...",1131660440785968503_1472830385,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472830385,1,1472830385
1,Organic Search,20160902,"{""browser"": ""Firefox"", ""browserVersion"": ""not ...",377306020877927890,"{""continent"": ""Oceania"", ""subContinent"": ""Aust...",377306020877927890_1472880147,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472880147,1,1472880147
2,Organic Search,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",3895546263509774583,"{""continent"": ""Europe"", ""subContinent"": ""South...",3895546263509774583_1472865386,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472865386,1,1472865386
3,Organic Search,20160902,"{""browser"": ""UC Browser"", ""browserVersion"": ""n...",4763447161404445595,"{""continent"": ""Asia"", ""subContinent"": ""Southea...",4763447161404445595_1472881213,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472881213,1,1472881213
4,Organic Search,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",27294437909732085,"{""continent"": ""Europe"", ""subContinent"": ""North...",27294437909732085_1472822600,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472822600,2,1472822600


In [6]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 903653 entries, 0 to 903652
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   channelGrouping       903653 non-null  object
 1   date                  903653 non-null  int64 
 2   device                903653 non-null  object
 3   fullVisitorId         903653 non-null  object
 4   geoNetwork            903653 non-null  object
 5   sessionId             903653 non-null  object
 6   socialEngagementType  903653 non-null  object
 7   totals                903653 non-null  object
 8   trafficSource         903653 non-null  object
 9   visitId               903653 non-null  int64 
 10  visitNumber           903653 non-null  int64 
 11  visitStartTime        903653 non-null  int64 
dtypes: int64(4), object(8)
memory usage: 82.7+ MB


In [7]:
columns = ['device', 'geoNetwork', 'totals', 'trafficSource']
def json_read(df):
    df = pd.read_csv(df, converters={column: json.loads for column in columns},dtype={'fullVisitorId': 'str'}) 
    for column in columns: 
        column_as_df = json_normalize(df[column]) 
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns] 
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    return df 

In [8]:
df = json_read("./train.csv")

In [9]:
# df.to_csv('255-df-format.csv')

In [10]:
# df_file = pd.read_csv('255-df-format.csv')

In [11]:
# df_file.head()

In [12]:
# df_file.shape

In [13]:
df # 900000 rows with 55 columns

,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,device.browserVersion,device.browserSize,device.operatingSystem,device.operatingSystemVersion,device.isMobile,device.mobileDeviceBranding,device.mobileDeviceModel,device.mobileInputSelector,device.mobileDeviceInfo,device.mobileDeviceMarketingName,device.flashVersion,device.language,device.screenColors,device.screenResolution,device.deviceCategory,geoNetwork.continent,geoNetwork.subContinent,geoNetwork.country,geoNetwork.region,geoNetwork.metro,geoNetwork.city,geoNetwork.cityId,geoNetwork.networkDomain,geoNetwork.latitude,geoNetwork.longitude,geoNetwork.networkLocation,totals.visits,totals.hits,totals.pageviews,totals.bounces,totals.newVisits,totals.transactionRevenue,trafficSource.campaign,trafficSource.source,trafficSource.medium,trafficSource.keyword,trafficSource.adwordsClickInfo.criteriaParameters,trafficSource.isTrueDirect,trafficSource.referralPath,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adContent,trafficSource.campaignCode
0,Organic Search,20160902,1131660440785968503,1131660440785968503_1472830385,Not Socially Engaged,1472830385,1,1472830385,Chrome,not available in demo dataset,not available in demo dataset,Windows,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,desktop,Asia,Western Asia,Turkey,Izmir,(not set),Izmir,not available in demo dataset,ttnet.com.tr,not available in demo dataset,not available in demo dataset,not available in demo dataset,1,1,1,1,1,NaN,(not set),google,organic,(not provided),not available in demo dataset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Organic Search,20160902,377306020877927890,377306020877927890_1472880147,Not Socially Engaged,1472880147,1,1472880147,Firefox,not available in demo dataset,not available in demo dataset,Macintosh,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,desktop,Oceania,Australasia,Australia,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,dodo.net.au,not available in demo dataset,not available in demo dataset,not available in demo dataset,1,1,1,1,1,NaN,(not set),google,organic,(not provided),not available in demo dataset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Organic Search,20160902,3895546263509774583,3895546263509774583_1472865386,Not Socially Engaged,1472865386,1,1472865386,Chrome,not available in demo dataset,not available in demo dataset,Windows,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,desktop,Europe,Southern Europe,Spain,Community of Madrid,(not set),Madrid,not available in demo dataset,unknown.unknown,not available in demo dataset,not available in demo dataset,not available in demo dataset,1,1,1,1,1,NaN,(not set),google,organic,(not provided),not available in demo dataset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Organic Search,20160902,4763447161404445595,4763447161404445595_1472881213,Not Socially Engaged,1472881213,1,1472881213,UC Browser,not available in demo dataset,not available in demo dataset,Linux,not available in demo dataset,False,not ava

In [14]:
df.isnull().sum()

channelGrouping                                           0
date                                                      0
fullVisitorId                                             0
sessionId                                                 0
socialEngagementType                                      0
visitId                                                   0
visitNumber                                               0
visitStartTime                                            0
device.browser                                            0
device.browserVersion                                     0
device.browserSize                                        0
device.operatingSystem                                    0
device.operatingSystemVersion                             0
device.isMobile                                           0
device.mobileDeviceBranding                               0
device.mobileDeviceModel                                  0
device.mobileInputSelector              

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 903653 entries, 0 to 903652
Data columns (total 55 columns):
 #   Column                                             Non-Null Count   Dtype 
---  ------                                             --------------   ----- 
 0   channelGrouping                                    903653 non-null  object
 1   date                                               903653 non-null  int64 
 2   fullVisitorId                                      903653 non-null  object
 3   sessionId                                          903653 non-null  object
 4   socialEngagementType                               903653 non-null  object
 5   visitId                                            903653 non-null  int64 
 6   visitNumber                                        903653 non-null  int64 
 7   visitStartTime                                     903653 non-null  int64 
 8   device.browser                                     903653 non-null  object
 9   devi

In [16]:
df['totals.pageviews'].mode()

0    1
dtype: object

In [17]:
columns_to_remove = [col for col in df.columns if df[col].nunique() == 1]
print("No. of variables with unique value: {}".format(len(columns_to_remove)))

No. of variables with unique value: 24


In [18]:
columns_to_remove

['socialEngagementType',
 'device.browserVersion',
 'device.browserSize',
 'device.operatingSystemVersion',
 'device.mobileDeviceBranding',
 'device.mobileDeviceModel',
 'device.mobileInputSelector',
 'device.mobileDeviceInfo',
 'device.mobileDeviceMarketingName',
 'device.flashVersion',
 'device.language',
 'device.screenColors',
 'device.screenResolution',
 'geoNetwork.cityId',
 'geoNetwork.latitude',
 'geoNetwork.longitude',
 'geoNetwork.networkLocation',
 'totals.visits',
 'totals.bounces',
 'totals.newVisits',
 'trafficSource.adwordsClickInfo.criteriaParameters',
 'trafficSource.isTrueDirect',
 'trafficSource.adwordsClickInfo.isVideoAd',
 'trafficSource.campaignCode']

In [19]:
for col in columns_to_remove:
    if set(['not available in demo dataset']) ==  set(df[col].unique()): continue
    print(col, df[col].dtypes, df[col].unique())

socialEngagementType object ['Not Socially Engaged']
totals.visits object ['1']
totals.bounces object ['1' nan]
totals.newVisits object ['1' nan]
trafficSource.isTrueDirect object [nan True]
trafficSource.adwordsClickInfo.isVideoAd object [nan False]
trafficSource.campaignCode object [nan '11251kjhkvahf']


In [20]:
# Creating another copy for filling missing values

In [21]:
df_1 = df.copy()

In [22]:
df_1.head()

,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,device.browserVersion,device.browserSize,device.operatingSystem,device.operatingSystemVersion,device.isMobile,device.mobileDeviceBranding,device.mobileDeviceModel,device.mobileInputSelector,device.mobileDeviceInfo,device.mobileDeviceMarketingName,device.flashVersion,device.language,device.screenColors,device.screenResolution,device.deviceCategory,geoNetwork.continent,geoNetwork.subContinent,geoNetwork.country,geoNetwork.region,geoNetwork.metro,geoNetwork.city,geoNetwork.cityId,geoNetwork.networkDomain,geoNetwork.latitude,geoNetwork.longitude,geoNetwork.networkLocation,totals.visits,totals.hits,totals.pageviews,totals.bounces,totals.newVisits,totals.transactionRevenue,trafficSource.campaign,trafficSource.source,trafficSource.medium,trafficSource.keyword,trafficSource.adwordsClickInfo.criteriaParameters,trafficSource.isTrueDirect,trafficSource.referralPath,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adContent,trafficSource.campaignCode
0,Organic Search,20160902,1131660440785968503,1131660440785968503_1472830385,Not Socially Engaged,1472830385,1,1472830385,Chrome,not available in demo dataset,not available in demo dataset,Windows,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,desktop,Asia,Western Asia,Turkey,Izmir,(not set),Izmir,not available in demo dataset,ttnet.com.tr,not available in demo dataset,not available in demo dataset,not available in demo dataset,1,1,1,1,1,NaN,(not set),google,organic,(not provided),not available in demo dataset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Organic Search,20160902,377306020877927890,377306020877927890_1472880147,Not Socially Engaged,1472880147,1,1472880147,Firefox,not available in demo dataset,not available in demo dataset,Macintosh,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,desktop,Oceania,Australasia,Australia,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,dodo.net.au,not available in demo dataset,not available in demo dataset,not available in demo dataset,1,1,1,1,1,NaN,(not set),google,organic,(not provided),not available in demo dataset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Organic Search,20160902,3895546263509774583,3895546263509774583_1472865386,Not Socially Engaged,1472865386,1,1472865386,Chrome,not available in demo dataset,not available in demo dataset,Windows,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,desktop,Europe,Southern Europe,Spain,Community of Madrid,(not set),Madrid,not available in demo dataset,unknown.unknown,not available in demo dataset,not available in demo dataset,not available in demo dataset,1,1,1,1,1,NaN,(not set),google,organic,(not provided),not available in demo dataset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Organic Search,20160902,4763447161404445595,4763447161404445595_1472881213,Not Socially Engaged,1472881213,1,1472881213,UC Browser,not available in demo dataset,not available in demo dataset,Linux,not available in demo dataset,False,not ava

In [23]:
df_1['totals.pageviews'].fillna(1, inplace=True) # mode of page views is 1
df_1['totals.newVisits'].fillna(0, inplace=True)
df_1['totals.bounces'].fillna(0, inplace=True) 
df_1["totals.transactionRevenue"].fillna(0.0, inplace=True)

In [24]:
#Changing objects to int
df_1['totals.pageviews'] = df_1['totals.pageviews'].astype(int)
df_1['totals.newVisits'] = df_1['totals.newVisits'].astype(int)
df_1['totals.bounces'] = df_1['totals.bounces'].astype(int)
df_1["totals.transactionRevenue"] = df_1["totals.transactionRevenue"].astype(float)

In [25]:
df_1['trafficSource.isTrueDirect'].fillna(False, inplace=True) 
df_1['trafficSource.adwordsClickInfo.isVideoAd'].fillna(True, inplace=True) # filling boolean with True


In [26]:
# df_1[df_1['geoNetwork.city'] == "(not available in demo dataset)"]['geoNetwork.city'] = np.nan
# df_1['geoNetwork.city'].fillna("NaN", inplace=True)

In [27]:
df_1.isnull().sum()

channelGrouping                                           0
date                                                      0
fullVisitorId                                             0
sessionId                                                 0
socialEngagementType                                      0
visitId                                                   0
visitNumber                                               0
visitStartTime                                            0
device.browser                                            0
device.browserVersion                                     0
device.browserSize                                        0
device.operatingSystem                                    0
device.operatingSystemVersion                             0
device.isMobile                                           0
device.mobileDeviceBranding                               0
device.mobileDeviceModel                                  0
device.mobileInputSelector              

In [28]:
columns = [col for col in df_1.columns if df_1[col].nunique() > 1]
df_1 = df_1[columns]

In [29]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 903653 entries, 0 to 903652
Data columns (total 35 columns):
 #   Column                                        Non-Null Count   Dtype  
---  ------                                        --------------   -----  
 0   channelGrouping                               903653 non-null  object 
 1   date                                          903653 non-null  int64  
 2   fullVisitorId                                 903653 non-null  object 
 3   sessionId                                     903653 non-null  object 
 4   visitId                                       903653 non-null  int64  
 5   visitNumber                                   903653 non-null  int64  
 6   visitStartTime                                903653 non-null  int64  
 7   device.browser                                903653 non-null  object 
 8   device.operatingSystem                        903653 non-null  object 
 9   device.isMobile                               90

In [30]:
df_1.isnull().sum()

channelGrouping                                      0
date                                                 0
fullVisitorId                                        0
sessionId                                            0
visitId                                              0
visitNumber                                          0
visitStartTime                                       0
device.browser                                       0
device.operatingSystem                               0
device.isMobile                                      0
device.deviceCategory                                0
geoNetwork.continent                                 0
geoNetwork.subContinent                              0
geoNetwork.country                                   0
geoNetwork.region                                    0
geoNetwork.metro                                     0
geoNetwork.city                                      0
geoNetwork.networkDomain                             0
totals.hit

In [31]:
# Removing columns with high null values 
df_1.drop(columns=['trafficSource.adContent', 'trafficSource.adwordsClickInfo.adNetworkType', 'trafficSource.adwordsClickInfo.gclId',
                'trafficSource.adwordsClickInfo.slot', 'trafficSource.adwordsClickInfo.page', 'trafficSource.referralPath',
                 'trafficSource.keyword'], inplace = True)

In [32]:
# for col in df_1.columns:
#     if len(df_1[col].unique()) == 1:
#         df_1.drop(col,inplace=True,axis=1)

In [33]:
print(df.shape)
print(df_1.shape)

(903653, 55)
(903653, 28)


In [34]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 903653 entries, 0 to 903652
Data columns (total 28 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   channelGrouping                           903653 non-null  object 
 1   date                                      903653 non-null  int64  
 2   fullVisitorId                             903653 non-null  object 
 3   sessionId                                 903653 non-null  object 
 4   visitId                                   903653 non-null  int64  
 5   visitNumber                               903653 non-null  int64  
 6   visitStartTime                            903653 non-null  int64  
 7   device.browser                            903653 non-null  object 
 8   device.operatingSystem                    903653 non-null  object 
 9   device.isMobile                           903653 non-null  bool   
 10  device.deviceCategor

In [35]:
# Converting to float
# df_1['fullVisitorId'] = df_1['fullVisitorId'].astype(float)
# df_1['sessionId'] = df_1['sessionId'].astype(float)

In [36]:
# df_1["date"] = pd.to_datetime(df_1["date"], format="%Y%m%d") # seting the column as pandas datetime
# df_1["weekday"] = df_1['date'].dt.weekday #extracting week day
# df_1["day"] = df_1['date'].dt.day # extracting day
# df_1["month"] = df_1['date'].dt.month # extracting day
# df_1["year"] = df_1['date'].dt.year # extracting day


In [37]:
df_1.shape

(903653, 28)

In [38]:
# Copy into another dataframe to complete feature Engineering

In [39]:
df_2 = df_1.copy()

In [40]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 903653 entries, 0 to 903652
Data columns (total 28 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   channelGrouping                           903653 non-null  object 
 1   date                                      903653 non-null  int64  
 2   fullVisitorId                             903653 non-null  object 
 3   sessionId                                 903653 non-null  object 
 4   visitId                                   903653 non-null  int64  
 5   visitNumber                               903653 non-null  int64  
 6   visitStartTime                            903653 non-null  int64  
 7   device.browser                            903653 non-null  object 
 8   device.operatingSystem                    903653 non-null  object 
 9   device.isMobile                           903653 non-null  bool   
 10  device.deviceCategor

In [41]:
# Visit time
df_2['diff_visitId_time'] = df_2['visitId'] - df_2['visitStartTime']
df_2['diff_visitId_time'] = (df_2['diff_visitId_time'] != 0).astype(int)
del df_2['visitId']

In [42]:
# Visit hour
df_2['formated_visitStartTime'] = df_2['visitStartTime'].apply(
    lambda x: time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(x)))
df_2['formated_visitStartTime'] = pd.to_datetime(df_2['formated_visitStartTime'])
df_2['visit_hour'] = df_2['formated_visitStartTime'].apply(lambda x: x.hour)
 
del df_2['visitStartTime']
del df_2['formated_visitStartTime']

In [43]:
# date
format_str = '%Y%m%d' 
df_2['formated_date'] = df_2['date'].apply(lambda x: datetime.strptime(str(x), format_str))
df_2['month'] = df_2['formated_date'].apply(lambda x:x.month)
df_2['quarter_month'] = df_2['formated_date'].apply(lambda x:x.day//8)
df_2['day'] = df_2['formated_date'].apply(lambda x:x.day)
df_2['weekday'] = df_2['formated_date'].apply(lambda x:x.weekday())
 
del df_2['date']

In [44]:
# total hits
df_2['totals.hits'] = df_2['totals.hits'].astype(int)
df_2['mean_hits_per_day'] = df_2.groupby(['day'])['totals.hits'].transform('mean')
del  df_2['day']

In [45]:
num_cols = ["totals.hits", "totals.pageviews", "visitNumber", 'totals.bounces',  'totals.newVisits']    
for col in num_cols:
    df_2[col] = df_2[col].astype(float)

In [46]:
# Removing unwanted id columns
df_2 = df_2.drop(['fullVisitorId','sessionId'], axis = 1)

In [47]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 903653 entries, 0 to 903652
Data columns (total 30 columns):
 #   Column                                    Non-Null Count   Dtype         
---  ------                                    --------------   -----         
 0   channelGrouping                           903653 non-null  object        
 1   visitNumber                               903653 non-null  float64       
 2   device.browser                            903653 non-null  object        
 3   device.operatingSystem                    903653 non-null  object        
 4   device.isMobile                           903653 non-null  bool          
 5   device.deviceCategory                     903653 non-null  object        
 6   geoNetwork.continent                      903653 non-null  object        
 7   geoNetwork.subContinent                   903653 non-null  object        
 8   geoNetwork.country                        903653 non-null  object        
 9   geoNetwork.regi

In [48]:
df_2.head()

,channelGrouping,visitNumber,device.browser,device.operatingSystem,device.isMobile,device.deviceCategory,geoNetwork.continent,geoNetwork.subContinent,geoNetwork.country,geoNetwork.region,geoNetwork.metro,geoNetwork.city,geoNetwork.networkDomain,totals.hits,totals.pageviews,totals.bounces,totals.newVisits,totals.transactionRevenue,trafficSource.campaign,trafficSource.source,trafficSource.medium,trafficSource.isTrueDirect,trafficSource.adwordsClickInfo.isVideoAd,diff_visitId_time,visit_hour,formated_date,month,quarter_month,weekday,mean_hits_per_day
0,Organic Search,1.0,Chrome,Windows,False,desktop,Asia,Western Asia,Turkey,Izmir,(not set),Izmir,ttnet.com.tr,1.0,1.0,1.0,1.0,0.0,(not set),google,organic,False,True,0,15,2016-09-02,9,0,4,4.638119
1,Organic Search,1.0,Firefox,Macintosh,False,desktop,Oceania,Australasia,Australia,not available in demo dataset,not available in demo dataset,not available in demo dataset,dodo.net.au,1.0,1.0,1.0,1.0,0.0,(not set),google,organic,False,True,0,5,2016-09-02,9,0,4,4.638119
2,Organic Search,1.0,Chrome,Windows,False,desktop,Europe,Southern Europe,Spain,Community of Madrid,(not set),Madrid,unknown.unknown,1.0,1.0,1.0,1.0,0.0,(not set),google,organic,False,True,0,1,2016-09-02,9,0,4,4.638119
3,Organic Search,1.0,UC Browser,Linux,False,desktop,Asia,Southeast Asia,Indonesia,not available in demo dataset,not available in demo dataset,not available in demo dataset,unknown.unknown,1.0,1.0,1.0,1.0,0.0,(not set),google,organic,False,True,0,5,2016-09-02,9,0,4,4.638119
4,Organic Search,2.0,Chrome,Android,True,mobile,Europe,Northern Europe,United Kingdom,not available in demo dataset,not available in demo dataset,not available in demo dataset,unknown.unknown,1.0,1.0,1.0,0.0,0.0,(not set),google,organic,True,True,0,13,2016-09-02,9,0,4,4.638119


In [49]:
# Copy for Encoding categorical features
df_3 = df_2.copy()

In [50]:
df_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 903653 entries, 0 to 903652
Data columns (total 30 columns):
 #   Column                                    Non-Null Count   Dtype         
---  ------                                    --------------   -----         
 0   channelGrouping                           903653 non-null  object        
 1   visitNumber                               903653 non-null  float64       
 2   device.browser                            903653 non-null  object        
 3   device.operatingSystem                    903653 non-null  object        
 4   device.isMobile                           903653 non-null  bool          
 5   device.deviceCategory                     903653 non-null  object        
 6   geoNetwork.continent                      903653 non-null  object        
 7   geoNetwork.subContinent                   903653 non-null  object        
 8   geoNetwork.country                        903653 non-null  object        
 9   geoNetwork.regi

In [51]:
df_3.head()

,channelGrouping,visitNumber,device.browser,device.operatingSystem,device.isMobile,device.deviceCategory,geoNetwork.continent,geoNetwork.subContinent,geoNetwork.country,geoNetwork.region,geoNetwork.metro,geoNetwork.city,geoNetwork.networkDomain,totals.hits,totals.pageviews,totals.bounces,totals.newVisits,totals.transactionRevenue,trafficSource.campaign,trafficSource.source,trafficSource.medium,trafficSource.isTrueDirect,trafficSource.adwordsClickInfo.isVideoAd,diff_visitId_time,visit_hour,formated_date,month,quarter_month,weekday,mean_hits_per_day
0,Organic Search,1.0,Chrome,Windows,False,desktop,Asia,Western Asia,Turkey,Izmir,(not set),Izmir,ttnet.com.tr,1.0,1.0,1.0,1.0,0.0,(not set),google,organic,False,True,0,15,2016-09-02,9,0,4,4.638119
1,Organic Search,1.0,Firefox,Macintosh,False,desktop,Oceania,Australasia,Australia,not available in demo dataset,not available in demo dataset,not available in demo dataset,dodo.net.au,1.0,1.0,1.0,1.0,0.0,(not set),google,organic,False,True,0,5,2016-09-02,9,0,4,4.638119
2,Organic Search,1.0,Chrome,Windows,False,desktop,Europe,Southern Europe,Spain,Community of Madrid,(not set),Madrid,unknown.unknown,1.0,1.0,1.0,1.0,0.0,(not set),google,organic,False,True,0,1,2016-09-02,9,0,4,4.638119
3,Organic Search,1.0,UC Browser,Linux,False,desktop,Asia,Southeast Asia,Indonesia,not available in demo dataset,not available in demo dataset,not available in demo dataset,unknown.unknown,1.0,1.0,1.0,1.0,0.0,(not set),google,organic,False,True,0,5,2016-09-02,9,0,4,4.638119
4,Organic Search,2.0,Chrome,Android,True,mobile,Europe,Northern Europe,United Kingdom,not available in demo dataset,not available in demo dataset,not available in demo dataset,unknown.unknown,1.0,1.0,1.0,0.0,0.0,(not set),google,organic,True,True,0,13,2016-09-02,9,0,4,4.638119


In [52]:
cat_cols = ["channelGrouping", "device.browser", 
             "device.operatingSystem", 'device.isMobile','device.deviceCategory',
            "geoNetwork.city", "geoNetwork.continent", 
            "geoNetwork.country", "geoNetwork.metro",
            "geoNetwork.networkDomain", "geoNetwork.region", 
            "geoNetwork.subContinent", 
             "trafficSource.campaign",
            "trafficSource.medium", 
             "trafficSource.source",
            'trafficSource.adwordsClickInfo.isVideoAd', 'trafficSource.isTrueDirect']

In [53]:
# Label encoding

In [54]:
for col in cat_cols:
    # print(col)
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(df_3[col].values.astype('str')))
    df_3[col] = lbl.transform(list(df_3[col].values.astype('str')))

In [55]:
# Converting to numerical columns


In [56]:
num_cols = ["totals.hits", "totals.pageviews", 
            "visitNumber", 
            'totals.bounces',  'totals.newVisits']    

for col in num_cols:
    df_3[col] = df_3[col].astype(float)

In [57]:
df_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 903653 entries, 0 to 903652
Data columns (total 30 columns):
 #   Column                                    Non-Null Count   Dtype         
---  ------                                    --------------   -----         
 0   channelGrouping                           903653 non-null  int64         
 1   visitNumber                               903653 non-null  float64       
 2   device.browser                            903653 non-null  int64         
 3   device.operatingSystem                    903653 non-null  int64         
 4   device.isMobile                           903653 non-null  int64         
 5   device.deviceCategory                     903653 non-null  int64         
 6   geoNetwork.continent                      903653 non-null  int64         
 7   geoNetwork.subContinent                   903653 non-null  int64         
 8   geoNetwork.country                        903653 non-null  int64         
 9   geoNetwork.regi

In [58]:
df_3.head()

,channelGrouping,visitNumber,device.browser,device.operatingSystem,device.isMobile,device.deviceCategory,geoNetwork.continent,geoNetwork.subContinent,geoNetwork.country,geoNetwork.region,geoNetwork.metro,geoNetwork.city,geoNetwork.networkDomain,totals.hits,totals.pageviews,totals.bounces,totals.newVisits,totals.transactionRevenue,trafficSource.campaign,trafficSource.source,trafficSource.medium,trafficSource.isTrueDirect,trafficSource.adwordsClickInfo.isVideoAd,diff_visitId_time,visit_hour,formated_date,month,quarter_month,weekday,mean_hits_per_day
0,4,1.0,11,16,0,0,3,21,204,150,0,258,24932,1.0,1.0,1.0,1.0,0.0,0,149,5,0,1,0,15,2016-09-02,9,0,4,4.638119
1,4,1.0,16,7,0,0,5,1,12,375,93,648,6673,1.0,1.0,1.0,1.0,0.0,0,149,5,0,1,0,5,2016-09-02,9,0,4,4.638119
2,4,1.0,11,16,0,0,4,19,181,78,0,325,25841,1.0,1.0,1.0,1.0,0.0,0,149,5,0,1,0,1,2016-09-02,9,0,4,4.638119
3,4,1.0,46,6,0,0,3,16,94,375,93,648,25841,1.0,1.0,1.0,1.0,0.0,0,149,5,0,1,0,5,2016-09-02,9,0,4,4.638119
4,4,2.0,11,1,1,1,4,13,211,375,93,648,25841,1.0,1.0,1.0,0.0,0.0,0,149,5,1,1,0,13,2016-09-02,9,0,4,4.638119


In [59]:
# http://blog.davidkaleko.com/feature-engineering-cyclical-features.html
# df_3['visit_hour_sin'] = np.sin(df_3.visit_hour*(2.*np.pi/24))
# df_3['visit_hour_cos'] = np.cos(df_3.visit_hour*(2.*np.pi/24))
# df_3['month_sin'] = np.sin((df_3.month-1)*(2.*np.pi/12))
# df_3['month_cos'] = np.cos((df_3.month-1)*(2.*np.pi/12))

In [60]:
# df_3['weekday_sin'] = np.sin((df_3.weekday-1)*(2.*np.pi/7))
# df_3['weekday_cos'] = np.cos((df_3.weekday-1)*(2.*np.pi/7))


In [61]:
# del df_3['month']
# del df_3['visit_hour']

In [62]:
# del df_3['weekday']

In [63]:
df_3.head()

,channelGrouping,visitNumber,device.browser,device.operatingSystem,device.isMobile,device.deviceCategory,geoNetwork.continent,geoNetwork.subContinent,geoNetwork.country,geoNetwork.region,geoNetwork.metro,geoNetwork.city,geoNetwork.networkDomain,totals.hits,totals.pageviews,totals.bounces,totals.newVisits,totals.transactionRevenue,trafficSource.campaign,trafficSource.source,trafficSource.medium,trafficSource.isTrueDirect,trafficSource.adwordsClickInfo.isVideoAd,diff_visitId_time,visit_hour,formated_date,month,quarter_month,weekday,mean_hits_per_day
0,4,1.0,11,16,0,0,3,21,204,150,0,258,24932,1.0,1.0,1.0,1.0,0.0,0,149,5,0,1,0,15,2016-09-02,9,0,4,4.638119
1,4,1.0,16,7,0,0,5,1,12,375,93,648,6673,1.0,1.0,1.0,1.0,0.0,0,149,5,0,1,0,5,2016-09-02,9,0,4,4.638119
2,4,1.0,11,16,0,0,4,19,181,78,0,325,25841,1.0,1.0,1.0,1.0,0.0,0,149,5,0,1,0,1,2016-09-02,9,0,4,4.638119
3,4,1.0,46,6,0,0,3,16,94,375,93,648,25841,1.0,1.0,1.0,1.0,0.0,0,149,5,0,1,0,5,2016-09-02,9,0,4,4.638119
4,4,2.0,11,1,1,1,4,13,211,375,93,648,25841,1.0,1.0,1.0,0.0,0.0,0,149,5,1,1,0,13,2016-09-02,9,0,4,4.638119


In [64]:
# cat_cols_1 = ['quarter_month','month','weekday','visit_hour']

In [65]:
# for col in cat_cols_1:
#     # print(col)
#     lbl = preprocessing.LabelEncoder()
#     lbl.fit(list(df_2[col].values.astype('str')))
#     df_2[col] = lbl.transform(list(df_2[col].values.astype('str')))

In [66]:
# df_2.info()

In [67]:
# plt.rcParams["figure.figsize"] = (30,30)

In [68]:
# sns.heatmap(df_3.corr(), annot = True)

In [69]:
# Feature Engineering completed

In [70]:
# Trying dimensionality reduction technique

In [71]:
df_3.head()

,channelGrouping,visitNumber,device.browser,device.operatingSystem,device.isMobile,device.deviceCategory,geoNetwork.continent,geoNetwork.subContinent,geoNetwork.country,geoNetwork.region,geoNetwork.metro,geoNetwork.city,geoNetwork.networkDomain,totals.hits,totals.pageviews,totals.bounces,totals.newVisits,totals.transactionRevenue,trafficSource.campaign,trafficSource.source,trafficSource.medium,trafficSource.isTrueDirect,trafficSource.adwordsClickInfo.isVideoAd,diff_visitId_time,visit_hour,formated_date,month,quarter_month,weekday,mean_hits_per_day
0,4,1.0,11,16,0,0,3,21,204,150,0,258,24932,1.0,1.0,1.0,1.0,0.0,0,149,5,0,1,0,15,2016-09-02,9,0,4,4.638119
1,4,1.0,16,7,0,0,5,1,12,375,93,648,6673,1.0,1.0,1.0,1.0,0.0,0,149,5,0,1,0,5,2016-09-02,9,0,4,4.638119
2,4,1.0,11,16,0,0,4,19,181,78,0,325,25841,1.0,1.0,1.0,1.0,0.0,0,149,5,0,1,0,1,2016-09-02,9,0,4,4.638119
3,4,1.0,46,6,0,0,3,16,94,375,93,648,25841,1.0,1.0,1.0,1.0,0.0,0,149,5,0,1,0,5,2016-09-02,9,0,4,4.638119
4,4,2.0,11,1,1,1,4,13,211,375,93,648,25841,1.0,1.0,1.0,0.0,0.0,0,149,5,1,1,0,13,2016-09-02,9,0,4,4.638119


In [72]:
df_3.shape

(903653, 30)

In [73]:
df_4 = df_3.copy()

In [74]:
df_4.head()

In [75]:
# Sort using date
df_4 = df_4.set_index(df_4['formated_date'])
df_4 = df_4.sort_index()

In [76]:
# df_4.head()

In [77]:
# Remove date column
df_4 = df_4.drop(['formated_date','totals.transactionRevenue'],axis=1)

In [78]:
# df_4.info()

In [79]:
# df_4.head()

In [80]:
pca = PCA(0.90)

In [81]:
pca.fit_transform(df_4)

In [82]:
# print(pd.DataFrame(pca.components_,columns=df_4.columns))
# No reduction in number of columns 

In [83]:
# df_4

In [84]:
# df_5 = pd.DataFrame(df_4)

In [85]:
# df_5.head()

In [86]:
# Dividing into training and testing

In [87]:
X = df_3.copy()

In [88]:
# X.shape

In [89]:
# X.head()

In [90]:
# X.info()

In [91]:
# X_new = X.drop('date', axis = 1)

In [92]:
from datetime import datetime, date

In [93]:
X = X.set_index(X['formated_date'])
X = X.sort_index()

In [94]:
# min(X['formated_date'])

In [95]:
# max(X['formated_date'])

In [96]:
train = X['2016-08-01':'2017-05-31']
test  = X['2017-05-31':]


In [97]:
print(train.shape)
print(test.shape)

(765707, 30)
(140238, 30)


In [98]:
X_train = train.drop(['formated_date','totals.transactionRevenue'], axis = 1)
X_test = test.drop(['formated_date','totals.transactionRevenue'], axis=1)

In [99]:
print(X_train.shape)
print(X_test.shape)

(765707, 28)
(140238, 28)


In [100]:
y_train = np.log1p(train["totals.transactionRevenue"].values)
y_test = np.log1p(test["totals.transactionRevenue"].values)

In [101]:
print(y_train.shape)
print(y_test.shape)

(765707,)
(140238,)


In [102]:
# # Dropping date and target column
# X_train = df_5.iloc[0:722922,:]
# X_test = df_5.iloc[722922:,:]

In [103]:
# print(X_train.shape)
# print(X_test.shape)

In [104]:
# X_train.head()

In [105]:
# out = test['totals.transactionRevenue'].values

In [106]:
# output = pd.DataFrame(out)

In [107]:
# output.columns = ['ActualRevenue']

In [108]:
# output.head()

In [109]:
# #  transaction revenue
# y = np.log1p(df_3["totals.transactionRevenue"].values)

In [110]:
# y.shape

In [111]:
# y_train = y[0:722922]
# y_test = y[722922:]

In [112]:
# print(y_train.shape)
# print(y_test.shape)

In [113]:
# pca.explained_variance_

In [114]:
# data_dmatrix = xgb.DMatrix(data=X_train,label=y_train)

In [115]:
# cv_results

In [116]:
# xg_reg = xgb.train(params=params, dtrain=data_dmatrix, num_boost_round=50)

In [117]:
# xgb_val_data = xgb.DMatrix(X_test, y_test)

In [118]:
# y_pred = xg_reg.predict(xgb_val_data)

In [119]:
# print(mean_squared_error(y_test, y_pred, squared=False))

In [120]:
# params = {"objective":"reg:squarederror",'colsample_bytree': 0.3,'learning_rate': 0.3,
#                 'max_depth': 5, 'alpha': 10}

# cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=3,
#                     num_boost_round=50,early_stopping_rounds=10,metrics="rmse", as_pandas=True, seed=123)

In [121]:
X_train.shape

(765707, 28)

In [122]:
from sklearn.preprocessing import StandardScaler

In [123]:
scaler = StandardScaler()

In [124]:
X_scale = scaler.fit_transform(X_train)

In [125]:
X_test_scale = scaler.fit_transform(X_test)

In [126]:
from sklearn.linear_model import RidgeCV
from sklearn.svm import LinearSVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor

In [127]:
# estimators = [
#      ('lr', RidgeCV()),
#      ('svr', LinearSVR(random_state=42))
#  ]
# reg = StackingRegressor(estimators=estimators,final_estimator=RandomForestRegressor(n_estimators=10,random_state=42))

In [128]:
# reg.fit(X_scale, y_train)

In [129]:
# y_pred = reg.predict(X_test_scale)

In [130]:
# print(mean_squared_error(y_test, y_pred, squared=False))

In [131]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR

In [132]:
print(X_train.shape)
print(y_train.shape)

(765707, 28)
(765707,)


In [ ]:
m = SVR()
m.fit(X_scale, y_train)
y_pred = m.predict(X_test_scale)

In [ ]:
mean_squared_error(y_test, y_pred)

In [ ]:
# param_grid = {'C': [0.1, 1, 10, 100, 1000], 
#               'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
#               'kernel': ['rbf']} 
  
# grid = GridSearchCV(SVR(), param_grid, verbose = 3)
  
# # fitting the model for grid search
# grid.fit(X_train, y_train)